In [16]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset, Sampler
from torchvision.datasets import DatasetFolder
from prefetch_generator import BackgroundGenerator

# This is for the progress bar.
from tqdm import tqdm

In [17]:
class DataLoaderX(DataLoader):
    def __iter__(self):
        return BackgroundGenerator(super().__iter__())

In [18]:
train_tfm = transforms.Compose([
    transforms.RandomResizedCrop((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5),
    transforms.RandomAffine(degrees=20, translate=(0.2, 0.2), scale=(0.7, 1.3)),
    transforms.ToTensor(),
])

test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [19]:
batch_size = 200

train_set = DatasetFolder('food-11/training/labeled', loader = lambda x : Image.open(x), extensions = 'jpg', transform = train_tfm)
valid_set = DatasetFolder('food-11/validation', loader = lambda x : Image.open(x), extensions = 'jpg', transform = test_tfm)
test_set = DatasetFolder('food-11/testing', loader = lambda x : Image.open(x), extensions = 'jpg', transform = test_tfm)
unlabeled_set = DatasetFolder('food-11/training/unlabeled', loader = lambda x : Image.open(x), extensions = 'jpg', transform = train_tfm)

train_loader = DataLoaderX(train_set, batch_size = batch_size, shuffle = True, num_workers = 0)
valid_loader = DataLoaderX(valid_set, batch_size = batch_size, shuffle = True, num_workers = 0)
test_loader = DataLoaderX(test_set, batch_size = batch_size, shuffle = False)

import gc
del valid_set, test_set
gc.collect()

38

In [20]:
# (x - kernels + 2 * padding) / strike + 1 
# (128 - 3 + 1) / 1 + 1 = 127
# (127 - 2 + 0) / 2 + 1 = 64
# (64 - 3 + 1) / 1 + 1 = 63
# (63 - 2 + 0) / 2 + 1 = 32
# (32 - 3 + 1) / 1 + 1 = 31
# (31 - 4 + 0) / 4 + 1 = 8

In [21]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            
            nn.AdaptiveAvgPool2d(output_size = 1),
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p = 0.6),
            
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p = 0.4),
            
            nn.Linear(64, 11),
        )
    
    def forward(self, x):
        x = self.cnn_layers(x)
        
        x = x.flatten(1)
        
        x = self.fc_layers(x)
        
        return x

In [22]:
class PseudoDataset(Dataset):
    def __init__(self, X, y):
        self.data = X
        #y = y.astype(int)
        self.label = y
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx][0], self.label[idx]

In [23]:
def get_pseudo_labels(dataset, model, threshold = 0.65):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    data_loader = DataLoaderX(dataset, batch_size = batch_size, shuffle = False)
    
    model.eval()
    
    cnt = 0
    idx = []
    labels = []
    softmax = nn.Softmax(dim = -1)
    
    for batch in tqdm(data_loader):
        imgs, _ = batch
        
        with torch.no_grad():
            logits = model(imgs.to(device))
            probs = softmax(logits)
        
        for prob in probs:
            val, cla = torch.max(prob, 0)
            if val.item() >= threshold:
                labels.append(cla.item())
                idx.append(cnt)
            cnt += 1
    dataset = PseudoDataset(Subset(dataset, idx), labels)
    
    model.train()
    return dataset  

In [24]:
class PropotionalSampler(Sampler):
    def __init__(self, concat_set, batch_size, minor_ratio, replacement = True, generator = None):
        self.dataset = concat_set
        self.batch_size = batch_size
        self.minor_ratio = minor_ratio
        self.sizes = concat_set.cumulative_sizes
        self.generator = generator
        self.replacement = replacement
    
    def __iter__(self):
        n1, n = self.sizes
        if self.generator is None:
            generator = torch.Generator()
            generator.manual_seed(int(torch.empty((), dtype = torch.int64).random_().item()))
        else:
            generator = self.generator
        
        if self.replacement:
            size_n1 = int(self.batch_size * (1 - self.minor_ratio))
            size_n2 = self.batch_size - size_n1
            
            for _ in range(int(np.ceil(n / self.batch_size))):
                idx_n1 = torch.randint(high = n1, size = (size_n1,), dtype = torch.int64, generator = generator).tolist()
                idx_n2 = torch.randint(low = n1, high = n, size = (size_n2,), dtype = torch.int64, generator = generator).tolist()
                idx_n1.extend(idx_n2)
                yield from idx_n1
        else:
            yield from torch.randperm(n, generator = generator).tolist()
    
    def __len__(self):
        return self.sizes[-1]

In [26]:
import torchvision
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#model = Classifier().to(device)
model_path = './model.ckpt'
#model.device = device

#model.load_state_dict(torch.load(model_path))

model = torchvision.models.resnet18(pretrained=False).to(device)
model.load_state_dict(torch.load('resnet18-f37072fd.pth'))
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 11).to(device)
model.device = device
for param in model.parameters():
    param.requires_grad = True
#model.load_state_dict(torch.load(model_path))

n_epochs = 60

do_semi = True
semi_turns = 10

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.00005, weight_decay = 1e-5)

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, eta_min = 1e-9, T_0 = 15)

best_acc = 0.0

threshold = 0.9

In [27]:
for epoch in range(n_epochs):
    if do_semi and best_acc > 0.8:
        pseudo_set = get_pseudo_labels(unlabeled_set, model, threshold)
        concat_set = ConcatDataset([train_set, pseudo_set])
        #sampler = PropotionalSampler(concat_set, batch_size = batch_size, minor_ratio = 0.9)
        train_loader = DataLoaderX(concat_set, batch_size = batch_size, num_workers = 0, shuffle = True)
    
    model.train()
    train_acc = []
    train_loss = []
    
    for batch in tqdm(train_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        
        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        
        optimizer.step()
        
        #scheduler.step()
        
        _, probs = torch.max(logits, 1)
        
        acc = (probs.cpu() == labels.cpu()).float().mean().item()
        train_acc.append(acc)
        train_loss.append(loss.item())
    
    train_acc = sum(train_acc) / len(train_acc)
    train_loss = sum(train_loss) / len(train_loss)
    
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    
    model.eval()
    valid_acc = []
    valid_loss = []
    
    for batch in tqdm(valid_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        
        with torch.no_grad():
            logits = model(imgs)
            loss = criterion(logits, labels)
        
        _, probs = torch.max(logits, 1)
        
        acc = (probs.cpu() == labels.cpu()).float().mean().item()
        valid_acc.append(acc)
        valid_loss.append(loss.item())
    
    valid_acc = sum(valid_acc) / len(valid_acc)
    valid_loss = sum(valid_loss) / len(valid_loss)
    
    if valid_acc > best_acc:
        best_acc = valid_acc
        torch.save(model.state_dict(), model_path)
    
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:15<00:00,  1.04it/s]


[ Train | 001/060 ] loss = 2.23189, acc = 0.24641


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.02it/s]


[ Valid | 001/060 ] loss = 1.78580, acc = 0.42208


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:15<00:00,  1.03it/s]


...

[ Valid | 059/060 ] loss = 0.57978, acc = 0.82625


100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:36<00:00,  1.01s/it]


[ Train | 060/060 ] loss = 0.29510, acc = 0.91328


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.09s/it]

[ Valid | 060/060 ] loss = 0.54202, acc = 0.84583


In [28]:
best_acc

0.8458333313465118

In [30]:
model = torchvision.models.resnet18(pretrained=False).to(device)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 11).to(device)
model.device = device
for param in model.parameters():
    param.requires_grad = True
model.load_state_dict(torch.load(model_path))

model.eval()

predictions = []

for batch in tqdm(test_loader):
    imgs, _ = batch
    imgs = imgs.to(device)
    
    with torch.no_grad():
        logits = model(imgs)
    
    _, probs = torch.max(logits, 1)
    
    predictions.extend(probs.cpu().numpy().tolist())

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:16<00:00,  1.01it/s]


In [31]:
with open("predict.csv", 'w') as f:
    f.write('Id,Category\n')
    
    for i, p in enumerate(predictions):
        f.write('{},{}\n'.format(i, p))

In [32]:
len(predictions)

3347